<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 بخش اول: استخراج داده
</font>
</h1>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در این پروژه به کار بر روی اطلاعات مرتبط با رمزارزها پرداخته خواهد شد و بدین منظور از داده‌های موجود در سایت CoinMarketCap استفاده خواهیم کرد. از آن‌جا که به دلیل داینامیک بودن صفحه‌ی نخست این وب‌سایت ممکن است استخراج داده‌ها از این صفحه با دشواری‌های بیشتری همراه باشد، قصد داریم از بخش تاریخچه‌ی آن استفاده کرده و اطلاعات رمزارزها را برای یک بازه‌ی زمانی مشخص استخراج کنیم.

</font>
</p>

In [ ]:
import selenium
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
import csv

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در این صفحه می‌توانید با انتخاب یک تاریخ مشخص، اطلاعات رمزارزها در آن روز را مشاهده کنید. ابتدا از شما می‌خواهیم لیست ۲۰۰ رمز ارز برتر (۲۰۰ سطر ابتدایی جدول) در تاریخ 2023/08/25 را استخراج کرده و داده‌های آن را به فرمت جدول آماده کنید.
</font>
</p>

In [ ]:
driver = webdriver.Chrome()
driver.get("https://coinmarketcap.com/historical/20230825/")

In [ ]:
num_scrolls = 30
scroll_increment = 300  # Adjust the size of each scroll increment (in pixels)
scroll_delay = 0.9 
for _ in range(num_scrolls):
    # Calculate the new scroll position
    current_scroll_position = driver.execute_script("return window.scrollY;")
    new_scroll_position = current_scroll_position + scroll_increment

    # Scroll to the new position
    driver.execute_script(f"window.scrollTo(0, {new_scroll_position});")

    # Wait for a short delay before the next scroll
    time.sleep(scroll_delay)

In [ ]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
elements = soup.find_all('div', class_='cmc-table__table-wrapper-outer')
div_element = elements[2]
table = div_element.find('table')
tbody = table.find('tbody')
rows = tbody.find_all('tr')[0:200]

In [ ]:
names = []
ranks = []
symbols = []
market_caps = []
prices = []
Circulating_Supplies = []
volumes = []
main_links =[]
historical_links = []

for row in rows:
    cells = row.find_all('td')
    # print(cells)
    rank = cells[0].text
    ranks.append(rank)
    
    name = cells[1].findAll('a')[1].text
    names.append(name)
    
    Symbol = cells[2].text
    symbols.append(Symbol)
    
    Market_Cap = cells[3].text
    market_caps.append(Market_Cap)
    
    Price = cells[4].text
    prices.append(Price)
    
    # Circulating_Supply	= cells[5].text
    # Circulating_Supplies.append(Circulating_Supply)
    
    volume24h = cells[6].text
    volumes.append(volume24h)
    
    href = row.find('a')['href']
    main_link = f"https://coinmarketcap.com{href}"
    main_links.append(main_link)
    
    historical_link = f"{main_link}historical-data"
    historical_links.append(historical_link)

In [ ]:
df_data = {
    "Rank" : ranks , 
    "Name" : names ,
    "Symbol" : symbols,
    "Market_Cap" : market_caps,
    "Price" : prices, 
    "volume24h" : volumes,
    "MainLink" : main_links,
    "HistoricalLink" : historical_links,
}

In [ ]:
df = pd.DataFrame(df_data)
df.to_csv('Top200Main.csv',index=False)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در صفحه‌ی تاریخچه‌ی هر ارز (مثل <a href="https://coinmarketcap.com/currencies/bitcoin/historical-data/">این لینک</a>) می‌توان اطلاعات گوناگونی از جمله موارد زیر را مشاهده کرد:<br>
<ul>
<li>Name: نام رمزارز</li>
<li>Symbol: نماد اختصاری رمزارز</li>
<li>Market Cap: ارزش کل در گردش بازار ارز مربوطه</li>
<li>Volume(24h): بیانگر این است که چقدر در ۲۴ ساعت اخیر این رمزارز در حال معامله شدن بوده</li>
<li>Price: قیمت ارز</li>
<li>Circulating supply: تعداد ارزی که در بازار به‌صورت عمومی در گردش‌ بوده (ضروری نیست)</li>
<li>High: بالاترین ارزش پولی ارز در بازه‌ی زمانی مربوطه</li>
<li>Low: پایین‌ترین ارزش پولی ارز در بازه‌ی زمانی مربوطه</li>
<li>Open:  ارزش پولی ارز در زمان شروع بازه‌ی زمانی معامله</li>
<li>Close: ارزش پولی ارز در زمان پایان شروع بازه‌ی زمانی معامله</li>
</ul>
</font>
</p>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
شما باید به‌ازای هر رمز ارز، اطلاعات ۳۶۵ روز اخیر آن را استخراج کنید. برای این‌کار می‌توانید از فایل‌های csv که خود سایت در اختیارتان می‌گذارد استفاده کنید و تنها بازه‌ی تاریخی را معادل ۳۶۵ روز اخیر (نسبت به لحظه‌ی استخراج داده‌ها) انتخاب کنید.<br>
</font>
</p>


In [ ]:
options = webdriver.EdgeOptions()
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')

driver = webdriver.Edge(options=options)
driver.get("https://coinmarketcap.com/historical/20230825/")

In [ ]:
# Scroll down the page in small steps to load more data
SCROLL_PAUSE_TIME = 1
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    for i in range(10):
        driver.execute_script(f"window.scrollBy(0, {last_height/10});")
        time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Get all the elements
Name = driver.find_elements(By.CLASS_NAME, "cmc-table__column-name--name.cmc-link")
Symbol = driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__symbol']/div")
MarketCap = driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__market-cap']/div")
Price = driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__price']/div")
CirculatingSupply= driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__circulating-supply']/div")
Volume =  driver.find_elements(By.CLASS_NAME,"cmc-link")

In [ ]:
# Write data to CSV file
with open('output.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Rank" , "Name", "Symbol", "MarketCap", "Price", "CirculatingSupply", "Volume"])
    for i in range(len(Name)):
        writer.writerow([i , Name[i].text, Symbol[i].text, MarketCap[i].text, Price[i].text, CirculatingSupply[i].text, Volume[i].text])

driver.quit()

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--start-fullscreen')

driver = webdriver.Chrome(options=options)

In [ ]:
# Scroll down the page in small steps to load more data
SCROLL_PAUSE_TIME = 30
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    for i in range(10):
        driver.execute_script(f"window.scrollBy(0, {last_height/10});")
        time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
driver.get("https://coinmarketcap.com/historical/20230825/")
time.sleep(5)
for i in range(1,201) :
    
    adad = str(i) 
    stt = "/html/body/div[1]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table/tbody/tr["+adad+"]/td[11]/div/div/div[1]"
    stlink = "/html/body/div[1]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table/tbody/tr["+adad+"]/td[11]/div/div/div[2]/ul/li[3]/a"
    button = driver.find_element(By.XPATH, stt)
    button.click()
    #button2 = driver.find_element(By.XPATH , "/html/body/div[1]/div[2]/div[2]/div/div[1]/h1")
    button2 = driver.find_element(By.XPATH ,stlink)
    time.sleep(2)
    button2.click()
    time.sleep(2)
    button1 = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[1]")
    button1.click()
    time.sleep(2)
    button2 = driver.find_element(By.XPATH , "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[2]/ul/li[5]")
    button2.click()
    time.sleep(2)
    button3 = driver.find_element(By.XPATH , "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div/div/div[2]/span/button")
    button3.click()
    time.sleep(2)
    button4 = driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[2]/div[1]/div")
    button4.click()
    time.sleep(3)
    driver.back()
    time.sleep(3)

    driver.execute_script("window.scrollBy(0, 47)")

In [ ]:
for name in upname:
    driver.get(f"https://coinmarketcap.com/currencies/{name}/historical-data/")
    time.sleep(2) 
    button1 = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[1]")
    button1.click()
    time.sleep(2)
    button2 = driver.find_element(By.XPATH , "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[2]/ul/li[5]")
    button2.click()
    time.sleep(2)
    button3 = driver.find_element(By.XPATH , "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div/div/div[2]/span/button")
    button3.click()
    time.sleep(2)
    button4 = driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[2]/div[1]/div")
    button4.click()
    time.sleep(10)
    
driver.quit()

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
علاوه بر این، به‌ازای هر رمز ارز نیاز است اطلاعاتی را از صفحه‌ی ویژه‌ی آن استخراج کنید. یکی از این اطلاعات مهم، لینک صفحه‌ی گیت‌هاب آن (در صورت وجود) است که در قسمت Official links قابل مشاهده است.
</font>
</p>

In [ ]:
driver = webdriver.Chrome()
driver.get("https://coinmarketcap.com/historical/20230825/")
driver.maximize_window()

In [ ]:
# Scroll down the page in small steps to load more data
SCROLL_PAUSE_TIME = 1
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    for i in range(10):
        driver.execute_script(f"window.scrollBy(0, {last_height/10});")
        time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
# Get all the elements
Name = driver.find_elements(By.CLASS_NAME, "cmc-table__column-name--name.cmc-link")
Symbol = driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__symbol']/div")
MarketCap = driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__market-cap']/div")
Price = driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__price']/div")
CirculatingSupply= driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__circulating-supply']/div")
Volume =  driver.find_elements(By.CLASS_NAME,"cmc-link")

# Scraping the Links of each Currency
Links = []
for name in Name:
    Links.append(name.get_attribute('href'))

In [ ]:
# Scraping the GitHub URL of each Currency
driver = webdriver.Chrome()
driver.maximize_window()

#Links = ['https://coinmarketcap.com/currencies/bitcoin/', 'https://coinmarketcap.com/currencies/tether/']

GitHubLink = []
for url in Links:
    driver.get(url)
    OfficialLinksTemplate = driver.find_elements(By.XPATH, "//a[@rel='nofollow noopener']")
    flag = 0
    for web in OfficialLinksTemplate:
            if web.text == "GitHub":
                GitHubLink.append(web.get_attribute('href'))
                flag = 1
    if flag == 0:
        GitHubLink.append("")

In [ ]:
# to CSV of GitHubLinks
with open('GitHubLinks.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Rank", "Name", "GitHub Link"])
    for i in range(len(GitHubLink)):
        writer.writerow([i+1 ,Name[i].text, GitHubLink[i]])
driver.quit()

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
مورد دیگر تگ‌هایی‌ست که در پایین همان بخش سمت چپ صفحه نوشته شده‌اند. به‌عنوان مثال برای بیت‌کوین سه تگ Mineable، PoW و SHA-256 نوشته شده است (استخراج تگ‌هایی که با کلیک بر روی دکمه‌ی show all نمایش داده می‌شوند بخش امتیازی خواهد بود).
</font>
</p>

In [ ]:
driver = webdriver.Chrome()
driver.get("https://coinmarketcap.com/historical/20230825/")
driver.maximize_window()

In [ ]:
# Scroll down the page in small steps to load more data
SCROLL_PAUSE_TIME = 1
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    for i in range(10):
        driver.execute_script(f"window.scrollBy(0, {last_height/10});")
        time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
# Get all the elements
Name = driver.find_elements(By.CLASS_NAME, "cmc-table__column-name--name.cmc-link")
Symbol = driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__symbol']/div")
MarketCap = driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__market-cap']/div")
Price = driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__price']/div")
CirculatingSupply= driver.find_elements(By.XPATH,"//td[@class='cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__circulating-supply']/div")
Volume =  driver.find_elements(By.CLASS_NAME,"cmc-link")

# Scraping the Links of each Currency
Links = []
for name in Name:
    Links.append(name.get_attribute('href'))

In [ ]:
# Scraping the Tag of each Currency

#Links = ['https://coinmarketcap.com/currencies/bitcoin/']
driver = webdriver.Edge()

Tag = []
for url in Links:
    driver.get(url)
    driver.maximize_window()
    button = driver.find_element(By.XPATH , "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div[2]/div[3]/section[2]/div/div[7]/div[1]/span")
    driver.execute_script("arguments[0].scrollIntoView();", button)
    driver.execute_script("window.scrollBy(0, 4000)")
    button2=driver.find_element(By.XPATH , "/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div[2]/div[3]/section[2]/div/div[7]/div[2]/div/span[4]")
    button2.click()
    time.sleep(2)

    TagTemplate = driver.find_elements(By.XPATH, "//div[@class='sc-16891c57-0 ddQhJW']/div/div/span/a")
    flag = 0
    TemporaryTag = []
    for tag in TagTemplate:
        print(tag.text)
        if tag.text != "":
            TemporaryTag.append(tag.text)
            flag = 1
    Tag.append(','.join(TemporaryTag))
    if flag == 0:
        Tag.append("")

In [ ]:
# to CSV of GitHubLinks
with open('Tags.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Rank", "Name", "Tags"])
    for i in range(len(Tag)):
        writer.writerow([i+1 ,Name[i].text, Tag[i]])
driver.quit()